In [1]:
! pip install -U transformers datasets evaluate
! pip install -U accelerate
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.1 MB/s eta 0:00:00


In [2]:
df = pd.read_csv(r'soc_202311261432.csv')
len(df)

59121

In [3]:
# df = df.sample(n=40000)
df = df.fillna('N/A')
df.head()

,CASE_ID,SOC_CODE,SOC_TITLE,JOB_DUTIES
0,21291992,53-3032.00,Heavy and Tractor-Trailer Truck Drivers,Drive a tractor-trailer combination or a truck...
1,21293592,37-3013.00,Tree Trimmers and Pruners,Trims and/or removes trees by climbing with a...
2,21291654,37-3011.00,Landscaping and Groundskeeping Workers,Landscape or maintain grounds using hand or po...
3,21286051,49-9045.00,"Refractory Materials Repairers, Except Brickma...",Reline or repair interior of refractory vessel...
4,21295310,37-3011.00,Landscaping and Groundskeeping Workers,Landscape or maintain properties using tools o...


In [4]:
# Assuming 'df' is your DataFrame with 'SOC Definition' and 'SOC Code' columns
# If you haven't split your data into training and testing sets yet, you can do it like this:
X_train, X_test, y_train, y_test = train_test_split(df['JOB_DUTIES'], df['SOC_CODE'], test_size=0.2, random_state=42)

In [5]:
label2id = {label: idx for idx, label in enumerate(df['SOC_CODE'].unique())}
id2label = {v: k for k, v in label2id.items()}

In [6]:
y_train = y_train.apply(lambda x: label2id[x])
y_test = y_test.apply(lambda x: label2id[x])

In [7]:
df_train = [{'text': text, 'label': label} for text, label in zip(X_train, y_train)]
df_test = [{'text': text, 'label': label} for text, label in zip(X_test, y_test)]

In [8]:
from datasets import DatasetDict, Dataset

# Convert the list of dictionaries to a dataset
ds_train = Dataset.from_dict({key: [item[key] for item in df_train] for key in df_train[0]})
ds_test = Dataset.from_dict({key: [item[key] for item in df_test] for key in df_test[0]})

df_train[0]

{'text': 'Tasks include: Operate vehicles and powered equipment, such as mowers, tractors, twin-axle vehicles, snow blowers, chainsaws, electric clippers, sod cutters, and pruning saws. Mow or edge lawns. Shovel snow and salt walks, driveways, or parking lots. Care for established lawns by mulching, aerating, weeding, grubbing, removing thatch, or trimming or edging around flower beds, walks, or walls.  Use hand tools, such as shovels, rakes, pruning saws, hedge or brush trimmers, or axes. Prune or trim trees, shrubs, or hedges. Gather and remove litter. Maintain or repair tools, equipment, or structures, such as building, greenhouses, fences, or benches, using hand or power tools. Mix and spray fertilizers, herbicides, or insecticides onto grass, shrubs, or trees.',
 'label': 2}

In [9]:
from transformers import AutoTokenizer

# Load pre-trained BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

In [11]:
# Tokenize input data
tokenized_ds_train = ds_train.map(preprocess_function, batched=True)
tokenized_ds_test = ds_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/47296 [00:00<?, ? examples/s]

Map:   0%|          | 0/11825 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(label2id), id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.2', '0.24.1')

In [17]:
tokenized_ds_train.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [18]:
training_args = TrainingArguments(
    output_dir='SOC_classification_model',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.961000,0.849349,0.823510
2,0.696300,0.714134,0.848710


TrainOutput(global_step=5912, training_loss=1.107523250966982, metrics={'train_runtime': 4793.6214, 'train_samples_per_second': 19.733, 'train_steps_per_second': 1.233, 'total_flos': 1.264610903064576e+16, 'train_loss': 1.107523250966982, 'epoch': 2.0})

In [19]:
model.save_pretrained('model')
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [20]:
from transformers import pipeline

text = "Drive a tractor-trailer combination or a truck with a capacity of at least 26,001 pounds. Secure cargo for transport, using ropes, blocks, chain, binders, or covers. Maneuver trucks into loading or unloading positions, following signals from loading crew and checking that vehicle and loading equipment are properly positioned. Perform basic vehicle maintenance tasks, such as adding oil, fuel, or radiator fluid or performing minor repairs. Load or unload trucks or help others with loading or unloading, using special loading-related equipment or other equipment as necessary. Remove debris from loaded trailers."

classifier = pipeline(task="text-classification", model="model", tokenizer='tokenizer')
classifier(text)

[{'label': '53-3032.00', 'score': 0.9769483208656311}]

In [ ]:
torch.save(model, open('torch_model.sav', 'wb'))
torch.save(tokenizer, open('torch_tokenizer.sav', 'wb'))